In [33]:
print ('Proyek Proyek Pertama : Membuat Model NLP dengan TensorFlow')
print ('Dataset Terlampir Bersama File zip')
print ('SMS Spam Collection Dataset : https://www.kaggle.com/uciml/sms-spam-collection-dataset')

Proyek Proyek Pertama : Membuat Model NLP dengan TensorFlow
Dataset Terlampir Bersama File zip
SMS Spam Collection Dataset : https://www.kaggle.com/uciml/sms-spam-collection-dataset


In [34]:
#Data Diri
print ('Nama  : Murwanjani Tejo Riyono')
print ('Kelas : MLFE M1')

Nama  : Murwanjani Tejo Riyono
Kelas : MLFE M1


In [35]:
import zipfile
local_zip = '/content/SMS_Spam_Collection_Dataset.zip' #Variable local_zip berisi direktori lokasi file zip disimpan

#Proses ekstrak file zip
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/SMS_Spam_Collection_Dataset')
zip_ref.close()

base_dir = '/content/SMS_Spam_Collection_Dataset/spam.csv' #Menentukan main/base direktori

In [36]:
import pandas as pd
import chardet

#Proses read file csv, merubah data menjadi dataframe dan menghapus kolom yang tidak diperlukan
with open(base_dir, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
df = pd.read_csv(base_dir,encoding='ISO-8859-1')
df= df.drop(columns='Unnamed: 2')
df= df.drop(columns='Unnamed: 3')
df= df.drop(columns='Unnamed: 4')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [37]:
#Cek Data
df.head() #Menampilkan 5 data awal

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [38]:
df.tail() #Menampilkan 5 data akhir

,v1,v2
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [39]:
#Import tensorflow dan test split library
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [40]:
# Proses one-hot-encoding karena data kategorikal dan membuat dataframe baru
category = pd.get_dummies(df.v1)
df_new = pd.concat([df, category], axis=1)
df_new = df_new.drop(columns='v1')

In [41]:
#Cek
df_new #Menampilkan 5 data awal&akhir pada dataframe baru

,v2,ham,spam
0,"Go until jurong point, crazy.. Available only ...",1,0
1,Ok lar... Joking wif u oni...,1,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,U dun say so early hor... U c already then say...,1,0
4,"Nah I don't think he goes to usf, he lives aro...",1,0
...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,0,1
5568,Will Ì_ b going to esplanade fr home?,1,0
5569,"Pity, * was in mood for that. So...any other s...",1,0
5570,The guy did some bitching but I acted like i'd...,1,0


In [42]:
#Mengubah nilai yang ada pada dataframe ke dalam tipe data numpy array menggunakan atribut values
teks = df_new['v2'].values
label = df_new[['ham', 'spam']].values

In [43]:
#Membagi data train dan data testing
teks_latih, teks_test, label_latih, label_test = train_test_split(teks, label, test_size=0.2)

In [44]:
#Proses tokenization dan konversi sample menjadi sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(teks_latih) 
tokenizer.fit_on_texts(teks_test)
 
sekuens_latih = tokenizer.texts_to_sequences(teks_latih)
sekuens_test = tokenizer.texts_to_sequences(teks_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [45]:
#Setting model (layer,Embedding, LSTM) dan fungsi compile
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
#Melatih model sebanyak 10x
model.fit(padded_latih, 
          label_latih, 
          epochs=10, 
          validation_data=(padded_test, label_test), 
          verbose=2)

Epoch 1/10
140/140 - 16s - loss: 0.2030 - accuracy: 0.9284 - val_loss: 0.0530 - val_accuracy: 0.9839
Epoch 2/10
140/140 - 13s - loss: 0.0307 - accuracy: 0.9906 - val_loss: 0.0441 - val_accuracy: 0.9874
Epoch 3/10
140/140 - 13s - loss: 0.0150 - accuracy: 0.9960 - val_loss: 0.0464 - val_accuracy: 0.9874
Epoch 4/10
140/140 - 13s - loss: 0.0071 - accuracy: 0.9982 - val_loss: 0.0459 - val_accuracy: 0.9910
Epoch 5/10
140/140 - 13s - loss: 0.0033 - accuracy: 0.9996 - val_loss: 0.0514 - val_accuracy: 0.9919
Epoch 6/10
140/140 - 13s - loss: 0.0019 - accuracy: 0.9998 - val_loss: 0.0649 - val_accuracy: 0.9901
Epoch 7/10
140/140 - 13s - loss: 0.0034 - accuracy: 0.9991 - val_loss: 0.0620 - val_accuracy: 0.9901
Epoch 8/10
140/140 - 13s - loss: 0.0021 - accuracy: 0.9996 - val_loss: 0.0648 - val_accuracy: 0.9928
Epoch 9/10
140/140 - 13s - loss: 0.0037 - accuracy: 0.9987 - val_loss: 0.0463 - val_accuracy: 0.9919
Epoch 10/10
140/140 - 13s - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.0657 - val_accura